In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pylab as plt
from astropy.table import Table

# Generate random data and add to catalog

In [ ]:
N_clusters = 1000
Data0 = Table({ 
    'RA': np.random.rand(N_clusters)*360,
    'DEC': np.random.rand(N_clusters)*180-90,
    'Z': .05+np.random.rand(N_clusters)*2,
    'RADIUS_ARCMIN': np.random.rand(N_clusters),
})
m00 = 13+2*np.random.rand(N_clusters*10)
prob_m = np.exp(-5*(m00-13)**2)
Data0['MASS'] = 10**np.random.choice(m00, N_clusters,
                                     p=prob_m/prob_m.sum(),
                                     replace=False)
Data0['MASS_ERR'] = Data0['MASS']*np.random.normal(loc=.2+0*Data0['MASS'], scale=.05)
Data0['Z_ERR'] = (1+Data0['Z'])*np.random.normal(loc=.03+0*Data0['Z'], scale=.005)

cls1 = np.random.choice(range(N_clusters), int(N_clusters*.7), replace=False)
input1 = Data0[cls1]
x = (Data0['MASS']/1e14)
comp = (x**3)/(x**3+1)
pur = (x)/(x+1)
prob = 1-pur
prob[cls1] = comp[cls1]
prob = prob/prob.sum()

input2 = Data0[np.random.choice(range(N_clusters), int(N_clusters*.7), replace=False, p=prob)]
input2['RA'] = np.random.normal(loc=input2['RA'], scale=1/3600.)
input2['DEC'] = np.random.normal(loc=input2['DEC'], scale=1/3600.)
input2['Z'] = np.random.normal(loc=input2['Z'], scale=.01)

In [ ]:
from clevar.catalog import ClData, Catalog

In [ ]:
c1 = Catalog('Cat1', ra=input1['RA'], dec=input1['DEC'], z=input1['Z'], mass=input1['MASS'],
            mass_err=input1['MASS_ERR'], z_err=input1['Z_ERR'])
c2 = Catalog('Cat2', ra=input2['RA'], dec=input2['DEC'], z=input2['Z'], mass=input2['MASS'],
            mass_err=input2['MASS_ERR'], z_err=input2['Z_ERR'])

# Match catalogs

In [ ]:
from clevar.match.proximity import ProximityMatch
from clevar.cosmology import AstroPyCosmology

In [ ]:
mt = ProximityMatch()

In [ ]:
mt_config1 = {'delta_z':.2,
            'match_radius': '1 mpc',
            'cosmo':AstroPyCosmology()}
mt_config2 = {'delta_z':.2,
            'match_radius': '10 arcsec'}
mt.prep_cat_for_match(c1, **mt_config1)
mt.prep_cat_for_match(c2, **mt_config2)

In [ ]:
%%time
mt.multiple(c1, c2)
mt.multiple(c2, c1)

In [ ]:
%%time
mt.unique(c1, c2, 'angular_proximity')
mt.unique(c2, c1, 'angular_proximity')

In [ ]:
mt.cross_match(c1)
mt.cross_match(c2)

# Recovery rate

In [ ]:
from clevar.metrics import recovery

In [ ]:
zbins = np.linspace(0, 2, 6)
mbins = np.logspace(13, 14, 5)

## Line type

In [ ]:
recovery.plot(c1, 'mt_cross', zbins, mbins)
plt.show()
recovery.plot(c1, 'mt_cross', zbins, mbins, shape='line')
plt.show()

## Transpose

In [ ]:
recovery.plot(c1, 'mt_cross', zbins, mbins,
              shape='line', transpose=True)

## Panels

In [ ]:
f, axes = recovery.plot_panel(c1, 'mt_cross', zbins, mbins)
plt.show()
f, axes = recovery.plot_panel(c1, 'mt_cross', zbins, mbins, transpose=True)
plt.show()

## 2D

In [ ]:
recovery.plot2D(c1, 'mt_cross', zbins, mbins)
plt.show()

recovery.plot2D(c1, 'mt_cross', zbins, mbins,
                add_num=True, num_kwargs={'fontsize':15})
plt.show()

# Distances of matching

In [ ]:
from clevar.metrics import distances

In [ ]:
ax = distances.central_position(c1, c2, 'mt_cross', 20, 'degrees')

In [ ]:
ax = distances.redshift(c1, c2, 'mt_cross', redshift_bins=20,
                        mass_bins=mbins, normalize='cat1')

# Scaling Relations

In [ ]:
from clevar.metrics import scaling

## Redshift plots

### Simple plot

In [ ]:
ax = scaling.redshift(c1, c2, 'mt_cross')
plt.show()
ax = scaling.redshift(c1, c2, 'mt_cross', add_err=True)
plt.show()

### Color points by $\log(M)$ value

In [ ]:
ax, cb = scaling.redshift_masscolor(c1, c2, 'mt_cross', add_err=True)

### Color points by density at plot

In [ ]:
ax, cb = scaling.redshift_density(c1, c2, 'mt_cross', add_err=True)

In [ ]:
ax, cb = scaling.redshift_density(c1, c2, 'mt_cross', ax_rotation=45)

### Split data into mass bins

In [ ]:
fig, axes = scaling.redshift_masspanel(c1, c2, 'mt_cross', add_err=True)

### Split data into mass bins and color by density

In [ ]:
fig, axes = scaling.redshift_density_masspanel(c1, c2, 'mt_cross', add_err=True)

## Mass plots

### Simple plot

In [ ]:
ax = scaling.mass(c1, c2, 'mt_cross', add_err=True)

### Color points by redshift value

In [ ]:
ax, cb = scaling.mass_zcolor(c1, c2, 'mt_cross', add_err=True)

### Color points by density at plot

In [ ]:
ax, cb = scaling.mass_density(c1, c2, 'mt_cross', add_err=True)

### Split data into redshift bins

In [ ]:
fig, axes = scaling.mass_zpanel(c1, c2, 'mt_cross', add_err=True)

### Split data into redshift bins and color by density

In [ ]:
fig, axes = scaling.mass_density_zpanel(c1, c2, 'mt_cross', add_err=True)